# Copyright 2026 Cognite AS

## Import the Libraries and Modules

In [ ]:
import sys
from pathlib import Path

utils = str(Path("../utils").resolve())
if utils not in sys.path:
    sys.path.append(utils)

from cognite_auth import interactive_client

## Create the Cognite Client

In [ ]:
client = interactive_client()

In [ ]:
from datetime import datetime, UTC

from cognite.client.data_classes.data_modeling.cdm.v1 import (
    CogniteTimeSeries,
    CogniteActivity,
    CogniteAsset,
)
from cognite.client.data_classes.data_modeling import (
    DirectRelationReference,
    NodeId,
    ViewId,
)
from cognite.client.data_classes import filters

### List data

Running the first command might open a new tab to verify the log in, if so, you can close that tab and come back to the notebook.

Below, we will look at how you can list different types of data from CDF. Note that when we list instances, we list all types of data from CDF, the `instance_type` tells us if it is a node or an edge. Unless we provide a source to retrieve properties from, the returned data is just the basic properties like space, external ID and timestamps corresponding to when the instance was created and last updated.

**Good to know**: You can use `list` when you want to run basic queries. Later, we'll take a look at the much more powerful `query` endpoint.

In [ ]:
client.data_modeling.instances.list(space="world_data", limit=10)

If we want to filter for specific `instance_type`: 
(here looking at edges, we can see the `start_node` and `end_note`)

In [ ]:
client.data_modeling.instances.list(space="world_data", instance_type='edge', limit=5)

#### List CogniteTimeSeries instances from the core data model

In [ ]:
tss = client.data_modeling.instances.list(
    sources=CogniteTimeSeries.get_source(),
    limit=5,  # or None for all
)

# Convert to pandas DataFrame
df = tss.to_pandas()
df

#### Listing CogniteActivities

In [ ]:
client.data_modeling.instances.list(sources=CogniteActivity.get_source(), limit=5)

## Searching in CDF

- Search the CDF resource types
- Filter the search results

### Exact search on one field (e.g. name, description etc)

In [ ]:
# In this exercise, we will search using the Cognite Asset view:
asset_view_id = CogniteAsset.get_source()

In [ ]:
res = client.data_modeling.instances.search(
    view=asset_view_id,
    query="islands",
    properties=["name"],
)
res.to_pandas()["name"]

### Search using multiple terms with at least one matching

By default, the search operator AND is used when multiple search terms are provided. Thus, we pass OR in the query below:

In [ ]:
res = client.data_modeling.instances.search(
    view=asset_view_id,
    query="Canada Norway Australia",
    properties=["name"],
    operator="OR",
)
res

#### Get all time series for the above asset

In [ ]:
time_series_view_id = CogniteTimeSeries.get_source()

res = client.data_modeling.instances.search(
    view=time_series_view_id,
    filter=filters.ContainsAny(
        # We use the time series view to reference the 'assets' property, which is a list of
        # direct relations to connected assets
        property=time_series_view_id.as_property_ref("assets"),
        values=[DirectRelationReference("world_data", "CAN")],
    )
)

res

## Retrieve data from CDF

### Retrieve datapoints from time series

In [ ]:
dps_list = client.time_series.data.retrieve(
    instance_id=[
        NodeId("world_data", "NRU"),
        NodeId("world_data", "ESP"),
    ],
    start=datetime(2000, 1, 1, tzinfo=UTC),
    end=datetime(2024, 1, 2, tzinfo=UTC),
    limit=10,
)

dps_list

#### Retrieve all items related to an asset

In [ ]:
# We have a node representing Europe:
europe = client.data_modeling.instances.retrieve_nodes(
    NodeId("world_data", "region:Europe"),
    node_cls=CogniteAsset,
)
# Let's check out what the parent of Europe is:
europe.parent

#### Let's now fetch the children of this asset (i.e. countries in Europe)

In [ ]:
countries = client.data_modeling.instances.list(
    sources=asset_view_id,
    filter=filters.Equals(
        property=asset_view_id.as_property_ref("parent"),
        value=europe.as_id(),
    ),
    limit=None,
)
len(countries)

#### Retrieve events ('activities')

In [ ]:
activity_view = CogniteActivity.get_source()

events = client.data_modeling.instances.search(
    view=activity_view,
    filter=filters.ContainsAny(
        property=activity_view.as_property_ref("assets"),
        values=[DirectRelationReference("world_data", "ITA")],
    ),
    limit=5,
)
events

#### Retrieve datapoints after 2020 for three countries

In [ ]:
italia_id = NodeId("world_data", "ITA")
norway_id = NodeId("world_data", "NOR")
france_id = NodeId("world_data", "FRA")

df = client.time_series.data.retrieve_dataframe(
    instance_id=[italia_id, norway_id, france_id],
    start=datetime(2020, 1, 1, tzinfo=UTC)
)
df

In [ ]:
# To extract the data for France, you can pass a NodeId:
df[france_id]

### Retrieve aggregated data using a granularity of 10 years

In [ ]:
df = client.time_series.data.retrieve_dataframe(
    instance_id=italia_id,
    aggregates=["average", "max", "min"],
    granularity="10y",
)
df

In [ ]:
# To extract the "max" aggregate, you can do the following:
df[italia_id, "max"]

### Retrieve latest datapoint before a particular time (last or any past time point)

In [ ]:
latest_dp = client.time_series.data.retrieve_latest(
    instance_id=norway_id,
    before="5w-ago",  # when not given, uses 'now'
)
latest_dp

### Plot datapoints
#### **Note**: Requires matplotlib to be installed

In [ ]:
client.time_series.data.retrieve(
    instance_id=[norway_id, italia_id, france_id]
).to_pandas().plot();